In [ ]:
function calculate_branching(x)
    machine = 0
    job = 0
    found = false
    for m in 1:D.m
        for j in 1:D.n
            if (mod(x[m][j],1) > ϵ)
                machine = m
                job = j
                found=true
                break
            end
        end
        found && break
    end
    return (machine,job)
end

In [ ]:
function solve_BP()
    global column_pool=Array{Array{Array{Int,1},1},1}(undef,D.m)
    # this is a 3-dimensional array
    # the first index is for machines
    # column_pool[i] contains the patterns for machine i which is a 2-dimensional array
    # column_pool[i][j] is a pattern  (1-dimensional array = vector)
    # pattern vectors are  such that:
    #   -the first element is the objective value contribution
    #   -the second to n+1 st elements are 1 or 0 depending on whether nth item is included in the pattern or not
    #   -the last element is the contribution to the convexity constraint (it is always 1)

    for i in 1:D.m
        column_pool[i]=Array{Array{Int,1},1}()
        push!(column_pool[i],vcat(10000,ones(Int,D.n+1))) # push an artificial pattern for each machine
        push!(column_pool[i],vcat(zeros(Int,D.n+1),1))    # push an empty pattern for each machine
    end

    # set the global lower and upper bound to +,- infinity
    global UB = Inf  # If we have a heuristic solution its value can be used   
    global LB = -Inf # If we have an initial relaxation bound it can be used 

    global Queue = Vector{Int}() # the Queue of nodes to process is an array of integers
    global tree = Vector{TreeNode}() # the branch and price tree is an array of TreeNode objects
    
    # the branch and price tree is initialized with the root node
    # the root node is its own parent, it has no children, its LB is -infinity, 
    # there are no variables set to 0 or 1 at the root node
    push!(tree,TreeNode(0,[],-Inf,[],[]))   
    push!(Queue,1)  # the indices of nodes to be processed are pushed to the Queue

    while length(Queue)>0
        println("Current list of nodes to be processed:", Queue)
        # we will process the last node in the Queue
        current = Queue[end]
        # Process_Node is the function that solves the restricted master LP
        # for the current node and returns its optimal solution
        # If the master LP is infeasible then returns []
        x = Process_Node(current)
        
        # if the current node is not infeasible and its lower
        # bound is promising then we branch
        if size(x,1)!=0 && tree[current].lb <= UB
            # in order to find a variable x_ij to branch on
            # we call the function calculate_branching()
            # it currently returns the first pair of (machine,job) indices
            # such that x_ij fractional
            # if no such pair is found then it returns (0,0)
            (machine,job) = calculate_branching(x)

            # if we have found a variable to branch on 
            # then we proceed with adding the children node
            # otherwise the optimal master LP solution is integer
            # there is no need for branching
            if (machine,job)!=(0,0)
                println("Two new nodes are created branching on variable x[$job,$machine]")
                # Noeud 1
                push!(tree,TreeNode(current,[],tree[current].lb,vcat((machine,job),tree[current].setzero),tree[current].setone))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
                # Noeud 2
                push!(tree,TreeNode(current,[],tree[current].lb,tree[current].setzero,vcat((machine,job),tree[current].setone)))
                push!(tree[current].children,length(tree))
                push!(Queue,length(tree))
            else
                println("The optimal relaxation solution was integer")
                println("Feasible solution with value $(tree[current].lb) found")
            end
        else
            println("The node is either infeasible or pruned by bound")
        end

        # calculate the global lower bound as the
        # minimum of lower bounds among all currently active nodes
        global LB = tree[current].lb
        for i in Queue
            if tree[i].lb <= LB global LB=tree[i].lb end
        end
        # print the current global lower and upper bounds
        println("LB=$LB,UB=$UB")
        # remove the processed node from the Queue
        deleteat!(Queue,findfirst(x -> x == current, Queue))
        # remove any nodes whose bound is not promising from the Queue
        deleteat!(Queue,unique(nodestobedeleted))
        # STOP if the optimality gap is sufficiently small
        if (2*(UB-LB)/(UB+LB))<=ϵ
            break
        end
    end
end